# 03 — Feature Engineering

Objectif : construire un jeu de données "prêt modélisation" à partir du dataset horaire final
(consommation + météo), en créant des variables temporelles, retardées et dérivées,
sans fuite d'information (time leakage), puis en définissant un split temporel
(train/validation/test).

**Entrée :** `data/processed/dataset_model_hourly.parquet`  
**Sortie :** `data/processed/dataset_features.parquet` (ou splits séparés)


In [31]:
from pathlib import Path
import numpy as np
import pandas as pd

PROJECT_ROOT = Path("/home/onyxia/france-grid-stress-prediction")
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

IN_PATH = DATA_PROCESSED / "dataset_model_hourly.parquet"
OUT_PATH = DATA_PROCESSED / "dataset_features.parquet"

assert IN_PATH.exists(), f"Missing input: {IN_PATH}"


In [32]:
df = pd.read_parquet(IN_PATH)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values("datetime").reset_index(drop=True)

df.head()


,datetime,load_mw,temperature_2m,wind_speed_10m,direct_radiation,diffuse_radiation,cloud_cover
0,2010-01-01 00:00:00,52685.0,4.273719,12.397994,0.0,0.0,93.96875
1,2010-01-01 01:00:00,52142.5,4.036219,12.709288,0.0,0.0,95.40625
2,2010-01-01 02:00:00,52081.5,3.812781,13.122019,0.0,0.0,96.46875
3,2010-01-01 03:00:00,52331.5,3.598719,13.308270,0.0,0.0,96.87500
4,2010-01-01 04:00:00,52171.0,3.426844,14.081800,0.0,0.0,94.78125


## Verification de cohérence

Vérifications minimales :
- période couverte (min/max)
- fréquence horaire (pas dominant)
- valeurs manquantes par colonne


In [33]:
print("Min datetime:", df["datetime"].min())
print("Max datetime:", df["datetime"].max())
print("N rows:", len(df))

# fréquence dominante
dt_diff = df["datetime"].diff().value_counts().head(5)
print("\nTop time diffs:")
print(dt_diff)

# NA par colonne
na_pct = (df.isna().mean() * 100).round(2).sort_values(ascending=False)
print("\nMissing values (%):")
print(na_pct.head(20))


Min datetime: 2010-01-01 00:00:00
Max datetime: 2024-12-31 23:00:00
N rows: 131496

Top time diffs:
datetime
0 days 01:00:00    131495
Name: count, dtype: int64

Missing values (%):
load_mw              3.92
datetime             0.00
temperature_2m       0.00
wind_speed_10m       0.00
direct_radiation     0.00
diffuse_radiation    0.00
cloud_cover          0.00
dtype: float64


## Définition de la cible

On commence avec une cible "à l’heure" : `y = load_mw`.
Ensuite, on pourra créer une variante H+24 : `y_h24 = load_mw(t+24)`.


In [34]:
TARGET_COL = "load_mw"
assert TARGET_COL in df.columns, f"Missing target column: {TARGET_COL}"

df["y"] = df[TARGET_COL].astype(float)

# df["y_h24"] = df["y"].shift(-24)


## Features calendaires

But : capturer les effets "humains" (heures, jours ouvrés/week-end) et la saisonnalité.


In [35]:
df["hour"] = df["datetime"].dt.hour
df["dayofweek"] = df["datetime"].dt.weekday  # 0=Monday
df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)
df["month"] = df["datetime"].dt.month
df["dayofyear"] = df["datetime"].dt.dayofyear

# Encodage cyclique (optionnel mais propre)
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["doy_sin"] = np.sin(2 * np.pi * df["dayofyear"] / 365.25)
df["doy_cos"] = np.cos(2 * np.pi * df["dayofyear"] / 365.25)


## Lags

But : donner au modèle l'inertie de la consommation (heure précédente, veille, semaine).
Attention : cela crée des NA au début de la série (normal).


In [ ]:
LAGS_H = [1, 24, 48, 168]

for lag in LAGS_H:
    df[f"load_lag_{lag}h"] = df["y"].shift(lag)


## Rolling statistics

But : fournir un contexte recent (niveau moyen, variabilité).
Fenêtres typiques : 24h, 7j.


In [ ]:
ROLL_WINDOWS = [24, 168]

for w in ROLL_WINDOWS:
    df[f"load_roll_mean_{w}h"] = df["y"].shift(1).rolling(w).mean()
    df[f"load_roll_std_{w}h"] = df["y"].shift(1).rolling(w).std()


## Features météo dérivées

On calcule des degrés-jours chauffage (HDD) et climatisation (CDD) si une température existe.
Adapte `TEMP_COL` selon le nom réel dans ton dataset.


In [ ]:
# Adapte ce nom à ta colonne température réelle (ex: "temperature", "temp", "t2m", etc.)
TEMP_COL_CANDIDATES = [c for c in df.columns if "temp" in c.lower()]
print("Temp candidates:", TEMP_COL_CANDIDATES[:10])

# Exemple : tu fixes explicitement la colonne
# TEMP_COL = "temp"
TEMP_COL = None

if TEMP_COL and TEMP_COL in df.columns:
    df["hdd_18"] = (18 - df[TEMP_COL]).clip(lower=0)
    df["cdd_22"] = (df[TEMP_COL] - 22).clip(lower=0)
